### 載入相關套件
<br>
pandas、numpy和re為必要，datetime暫時不需要，time則為選配

In [161]:
import pandas as pd
import numpy as np
import re
import datetime  #將變項轉為時間格式，目前不需要
import time  #輸出檔案時加註時間

### 讀取檔案
#### 所需檔案清單

- [ ] GRB爬蟲結果
- [ ] MOST爬蟲結果
- [ ] 科系地址清單
- [ ] 學門清單
- [ ] 名冊

In [165]:
GRB_path = r'C:\Users\user\Desktop\py\TEST\GRB_list_text_crawl__0817_1252.xlsx'  #GRB爬蟲結果路徑
MOST_path = r'C:\Users\user\Desktop\py\TEST\MOST_list_text_crawl__0817_1019.xlsx'  #MOST爬蟲結果路徑
address_list_path = r'C:\Users\user\Desktop\py\merge\finaltest\科系地址.xlsx'  #科系地址路徑
discipline_path = r'C:\Users\user\Desktop\py\merge\finaltest\學門架構與研究領域.xlsx'  #學門清單路徑
major_lsit_path = r'C:\Users\user\Desktop\py\merge\108調查計畫名冊_全司1081122.xlsx'  #名冊路徑

GRB = pd.DataFrame(pd.read_excel(GRB_path))  #GRB名冊
MOST = pd.DataFrame(pd.read_excel(MOST_path))  #MOST爬蟲結果
address_list = pd.DataFrame(pd.read_excel(address_list_path))  #科系地址
discipline_lsit = pd.DataFrame(pd.read_excel(discipline_path))  #學門清單
major_lsit = pd.DataFrame(pd.read_excel(major_lsit_path))  #名冊

print("GRB清單有"+str(len(GRB))+"筆資料")
print("MOST清單有"+str(len(MOST))+"筆資料")
print("原始清單有"+str(len(major_lsit))+"筆資料")

GRB清單有217筆資料
MOST清單有216筆資料
原始清單有216筆資料


C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


### 修改變項名稱
<br>
前面增加檔案名稱，比較方便判別來源<br>
待最終步驟再一次修改變項名稱，並選擇保留那些變項



In [166]:
address_list.columns = ['ADD學校名稱', 'ADD系所名稱', 'ADD機關地址']

MOST.columns = [
    'MOST計畫主持人', 'MOST執行起訖', 'MOST機關名稱', 'MOST調查計畫全稱(中)', 'MOST關鍵字(中文)',
    'MOST關鍵字(英文)', 'MOST摘要(中文)', 'MOST摘要(英文)', 'MOST缺失情況'
]

GRB.columns = [
    'GRB全部計畫主持人', 'GRB計畫編號', 'GRB執行機構', 'GRB調查計畫全稱(中)', 'GRB調查計畫全稱(英)', 'GRB研究領域_GRB',
    'GRB核定金額', 'GRB研究期間','GRB關鍵字(中文)', 'GRB關鍵字(英文)', 'GRB摘要(中文)', 'GRB摘要(英文)',
    'GRB網站依據(缺失情況)'
]

### 將名冊的時間轉換格式
**python不認識民國時間，所以要自己轉**

**原始名冊**的格式為  民國年/月份/日期  <br>
 > 109/08/01<br>

而**MOST爬蟲**結果格式為
 > 2021-08-01
 
後續要比對時間，所以統一轉成和MOST相同

In [167]:
d_star = major_lsit['執行期限(起)']
for i in range(len(major_lsit)):
    d_star.iloc[i] = d_star.iloc[i].replace(
        d_star.iloc[i][0:3], str(int(d_star.iloc[i][0:3]) + 1911))

d_end = major_lsit['執行期限(迄)']
for i in range(len(major_lsit)):
    d_end.iloc[i] = d_end.iloc[i].replace(d_end.iloc[i][0:3],
                                          str(int(d_end.iloc[i][0:3]) + 1911))

    
    
major_lsit['執行期限(起)'] = major_lsit['執行期限(起)'].str.replace('/', '-')
major_lsit['執行期限(迄)'] = major_lsit['執行期限(迄)'].str.replace('/', '-')



'''
d=pd.to_datetime(d,format='%Y/%m/%d')
'''
'''
# 並將「日期」列設置為索引。
cluster_data['date'] = pd.to_datetime(cluster_data['date']) 
cluster_data = cluster_data.set_index(['date'], drop=True)
cluster_data
'''

C:\Users\user\AppData\Local\Temp\ipykernel_12568\603121648.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_star.iloc[i] = d_star.iloc[i].replace(
C:\Users\user\AppData\Local\Temp\ipykernel_12568\603121648.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_end.iloc[i] = d_end.iloc[i].replace(d_end.iloc[i][0:3],


"\n# 並將「日期」列設置為索引。\ncluster_data['date'] = pd.to_datetime(cluster_data['date']) \ncluster_data = cluster_data.set_index(['date'], drop=True)\ncluster_data\n"

### 處理學門代號
<br>

**原始名冊**的學門格式為    <br>
 > H41A4-策略管理

使用將其拆為
 > H41A4    <br> 
            <br>
 -策略管理

取前半部分 H41A4 做為「學門代號」之變項<br>
並確保 **原始名冊** 和 **學門清單** 其頭尾沒有空格


In [168]:
major_lsit['學門代號'] = major_lsit['學門'].str.split('-').str.get(0)

major_lsit['學門代號'] = major_lsit['學門代號'].str.strip()
discipline_lsit['學門代碼\n(第3層)'] = discipline_lsit['學門代碼\n(第3層)'].str.strip()

### 第一次合併 Merge1
<br>

<font color=#FF0000>**橫向**</font>合併  **原始名冊** 和  **學門清單** 

primary key為  **學門代碼**

以**原始名冊**為主，Left join **學門清單**


![Leftjoin](https://upload.cc/i1/2022/08/09/5FZAqy.png "Left join")



以下示意圖


| 原始名冊       | <font color=#0000FF>學門代碼</font>  | 學門清單      |
|-              | -                                   |-              |
| 內容          | 內容                                 | 內容          |
| 內容          | 內容                                 | 內容          |

<br>

Merge1 = 原始名冊 +  學門清單

In [169]:
major_lsit['學門'] = major_lsit['學門'].str.strip()
discipline_lsit['代碼加學門'] = discipline_lsit['代碼加學門'].str.strip()

merge1 = pd.merge(major_lsit,
                  discipline_lsit,
                  left_on='學門',
                  right_on='代碼加學門',
                  how='left')

print("merge1有"+str(len(merge1))+"筆資料")

merge1有216筆資料


### 第二次合併 Merge2
<br>

<font color=#FF0000>**橫向**</font>合併  **merge1** 和  **科系地址** 

primary key為  **機關名稱(系所名稱)**

以**merge1**為主，Left join **科系地址**

![Leftjoin](https://upload.cc/i1/2022/08/10/jH3scl.png "Left join")


Merge2 = 原始名冊 +  學門清單 + 科系地址

In [170]:
merge2 = pd.merge(merge1,
                  address_list,
                  left_on='機關名稱'.strip(),
                  right_on='ADD系所名稱'.strip(),
                  how='left')

print("merge2有"+str(len(merge2))+"筆資料")

merge2有216筆資料


### 針對計畫編號進行處理
<br>


 > MOST109-2410-H160-<font color=#0000FF>09</font>-SS3 

從原始清單中有發現到以下問題

- problem 1-- 藍字變項部份會有**未滿百位**的情況，如：09，而非009
- problem 2-- 原始清單的編號是拆開的，如下表

|計畫主持人|補|助|計|畫|<font color=#0000FF>編</font>|號|計畫名稱|
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|翁一珍|109|2410|H|160|<font color=#0000FF>09</font>|SS3|台灣高等教育專業服務學習方案效益評估的多元觀點

因此處理方法為

- 自動補齊變項「編」到百位
- 將各個變項數值轉為字串(str)，再用map()函數映射到新的變項-「原計畫編號」，添加「-」符合格式


|原計畫編號|
|:---------:|
|MOST109-2410-H160-<font color=#0000FF>009</font>-SS3|


In [171]:
merge2['編'] = merge2['編'].apply(lambda x: str(x).zfill(3))   #---補到百位

merge2['原計畫編號'] = 'MOST' + merge2['補'].map(str) + '-' + merge2['助'].map(
    str) + '-' + merge2['計'].map(str) + merge2['畫'].map(
        str) + '-' + merge2['編'].map(str) + '-' + merge2['號'].map(str)    #---把編號組在一起
merge2['原計畫編號'] = merge2['原計畫編號'].str.strip()



SaveFile_Path = r'C:\Users\user\Desktop\py\TEST'  #要儲存檔案的路徑

merge2_Name = r'merge2'  #儲存的檔名，可以不用更動



merge2.to_excel(SaveFile_Path + '\\' + merge2_Name + "_" + "_" +
                      time.strftime("%m%d") + "_" + time.strftime("%H%M") +
                      ".xlsx",
                      encoding="utf_8_sig",
                      index=False)

### 第三次合併 Merge3
<br>

先把GRB的作者切成

-  計畫主持人
-  共同/協同主持人

此次合併將GRB和MOST的爬蟲資料對應

primary key為  **計畫主持人**

**Merge3可視為爬蟲資料的大合集**

以資料較多的GRB為主，Left join MOST檔案

| GRB| MOST| 
|-   |-    |
|內容| 內容| 
|內容| 內容| 


Merge3 = GRB + MOST

In [172]:
en = re.compile(r'[a-zA-Z\-\；\;]')
cn = re.compile(r'[\u4e00-\u9fa5\；\;]')

for i in GRB.index:
    GRB.loc[i,'GRB全部主持人中文姓名'] = "".join(cn.findall(GRB.loc[i, 'GRB全部計畫主持人']))
    GRB.loc[i,'GRB全部計畫主持人英文姓名'] = "".join(en.findall(GRB.loc[i, 'GRB全部計畫主持人']))

GRB['GRB計畫主持人'] = GRB['GRB全部主持人中文姓名'].str.split(';',1).str.get(0).replace(';;;;;;', '').replace(';;', '')
GRB['共同/協同主持人'] = GRB['GRB全部主持人中文姓名'].str.split(';',1).str.get(1).replace(';;;;;;', '').replace(';;', '')
GRB['計畫主持人英文姓名'] = GRB['GRB全部計畫主持人英文姓名'].str.split(';',1).str.get(0).replace(';;;;;;', '').replace(';;', '')
GRB['共同/協同主持人英文姓名'] = GRB['GRB全部計畫主持人英文姓名'].str.split(';',1).str.get(1).replace(';;;;;;', '').replace(';;', '')
GRB['GRB計畫編號'] = GRB['GRB計畫編號'].str.strip()


GRB = GRB.drop(['GRB全部主持人中文姓名'], axis=1)
GRB = GRB.drop(['GRB全部計畫主持人英文姓名'], axis=1)

'''
merge3 = pd.merge(GRB,
                  MOST,
                  left_on='GRB計畫主持人'.strip(),
                  right_on='MOST計畫主持人'.strip(),
                  how='left')
'''
merge3 = GRB.join(MOST)


'''

merge3 = merge3.drop_duplicates(subset='MOST調查計畫全稱(中)',
                               keep='first',
                               inplace=False,
                               ignore_index=False)
'''



print("merge3有"+str(len(merge3))+"筆資料")


SaveFile_Path = r'C:\Users\user\Desktop\py\TEST'  #要儲存檔案的路徑

error_Name = r'merge3'  #儲存的檔名，可以不用更動



merge3.to_excel(SaveFile_Path + '\\' + error_Name + "_" + "_" +
                      time.strftime("%m%d") + "_" + time.strftime("%H%M") +
                      ".xlsx",
                      encoding="utf_8_sig",
                      index=False)


merge3有217筆資料


### 最終合併 final
<br>

將merge2 和 merge3合併

primary key為  **計畫主持人**

以包含原始清單的merge2 為主，Left join merge3



final = 原始名冊 + 學門清單 + 科系地址 + GRB + MOST

**最終資料示意圖**

|原始名冊|學門清單|科系地址|GRB|MOST|
|:-:|:-:|:-:|:-:|:-:|  
|內容|內容|內容|內容|內容|
|內容|內容|內容|內容|內容|

In [173]:
'''

final = pd.merge(merge2,
                 merge3,
                 left_on='計畫主持人'.strip(),
                 right_on='MOST計畫主持人'.strip(),
                 how='left')
'''



final =merge2.join(merge3)

final_Name = r'final'  #儲存的檔名，可以不用更動


final.to_excel(SaveFile_Path + '\\' + final_Name + "_" + "_" +
                      time.strftime("%m%d") + "_" + time.strftime("%H%M") +
                      ".xlsx",
                      encoding="utf_8_sig",
                      index=False)

print("final有"+str(len(final))+"筆資料")

final有216筆資料


## 針對最終檔案進行資料處理
<br>



### 分割計畫執行期間
<br>

**計畫執行期間以MOST為主**，因此直接分割MOST執行起訖

 >2021-08-01~2022-07-31
 
 使用 .str.split ('~') 
 <br>
 
 將執行期間從 '~' 分割成兩部份
 
 >2021-08-01, 2022-07-31
 
使用.str.get(0) ←輸入0 取前半段 當作 計畫起始時間 <br>
使用.str.get(1) ←輸入1 取後半段 當作 計畫結束時間
 <br>

**此步驟沒有取代原始的「MOST執行起訖」變項，而是分割出兩個新的**
 <br>


|MOST執行起訖|計畫執行期間起|計畫執行期間迄|
|:-:|:-:|:-:|
|2021-08-01~2022-07-31|2021-08-01|2022-07-31|



In [174]:
final['MOST計畫執行期間起'] = final['MOST執行起訖'].str.split('~').str.get(0)
final['MOST計畫執行期間迄'] = final['MOST執行起訖'].str.split('~').str.get(1)
'''
result['計畫執行期間起'] = pd.to_datetime(result['計畫執行期間起'])
result['計畫執行期間迄'] = pd.to_datetime(result['計畫執行期間迄'])
'''

"\nresult['計畫執行期間起'] = pd.to_datetime(result['計畫執行期間起'])\nresult['計畫執行期間迄'] = pd.to_datetime(result['計畫執行期間迄'])\n"

### 執行年期轉換
<br>

>~~MOST109-2410-H003-062-SS~~<font color=#0000FF>S</font><br>
>~~MOST109-2410-H029-027-SS~~<font color=#0000FF>2</font><br>
>~~MOST109-2423-H037-001-SS~~<font color=#0000FF>3</font><br>

從編號可以觀察到，除了1年期為SSS，其餘皆為年份數字<br>
因此，將編號從'**SS**'切割，取後半部<br>
把'S'轉為1<br>
得到以下結果<br>

>S －轉換－→ 1<br>
>2<br>
>3<br>




In [175]:
final['原計畫編號'].str.split('SS').str.get(1)
final['年期'] = final['原計畫編號'].str.split('SS').str.get(1).str.replace('S', '1')

### 刪除重複計畫
<br>

原因在於GRB使用excel匯出版本，會有不同年份的計畫<br>

使用 keep='first' 保留第一筆(年份最小)

In [176]:
final = final.drop_duplicates(subset='計畫名稱',
                              keep='first',
                              inplace=False,
                              ignore_index=False)

### 計畫編號比對
<br>

將原始清單之編號與GRB編號進行比對<br>


> if original_num == GRB_num:<br>
    >&emsp;&emsp;return Ture<br>
> else:<br>
    >&emsp;&emsp;return False<br>
    

再用map()映射，把'Ture'轉成「編號一致」；把'False'轉成「編號不一致」<br>

起始和結束日期也是同樣方法<br>

最後把存放Ture、False的變項刪除<br>

In [177]:

#----------------------------------------------------編號

original_num = final['原計畫編號']
GRB_num = final['GRB計畫編號']

original_com = pd.DataFrame(original_num)
GRB_com = pd.DataFrame(GRB_num)

final['比對'] = (original_num == GRB_num)
final['編號比對結果'] = final['比對'].map({True: '編號一致', False: '編號不一致'})

#----------------------------------------------------起始

major_start_com = final['執行期限(起)']
most_start_com = final['MOST計畫執行期間起']

Dmajor_start_com = pd.DataFrame(major_start_com)
Dmost_start_com = pd.DataFrame(most_start_com)
final['起始日期比對'] = (major_start_com == most_start_com)
final['起始日期比對結果'] = final['起始日期比對'].map({True: '日期一致', False: '日期不一致'})

#----------------------------------------------------結束
major_end_com = final['執行期限(迄)']
most_end_com = final['MOST計畫執行期間迄']

Dmajor_end_com = pd.DataFrame(major_end_com)
Dmost_end_com = pd.DataFrame(most_end_com)
final['結束日期比對'] = (major_end_com == most_end_com)
final['結束日期比對結果'] = final['結束日期比對'].map({True: '日期一致', False: '日期不一致'})

'''
final = final.drop(['比對'], axis=1)
final = final.drop(['起始日期比對'], axis=1)
final = final.drop(['結束日期比對'], axis=1)
'''

final['比對']

0      False
1      False
2      False
3      False
4       True
       ...  
211    False
212    False
213    False
214    False
215    False
Name: 比對, Length: 216, dtype: bool

### 用GRB替補MOST
<br>

以MOST為主，但若MOST有遺漏資訊，使用GRB替補

先把NaN空值轉為 None<br>
建立一個額外的DataFrame (aaa)<br>
把MOST和GRB的資訊放在一起，前面GRB；後面MOST<br>
只要MOST欄位有空值，則向前替補一行 limit=1 <br>


替補示意圖

|GRB關鍵字(中文)|MOST關鍵字(中文)|GRB關鍵字(英文)|MOST關鍵字(英文)|GRB摘要(中文)|MOST摘要(中文)|GRB摘要(英文)|MOST摘要(英文)|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|A|無|B|無|無|無|D|無|

<br>
<br>

|GRB關鍵字(中文)|MOST關鍵字(中文)|GRB關鍵字(英文)|MOST關鍵字(英文)|GRB摘要(中文)|MOST摘要(中文)|GRB摘要(英文)|MOST摘要(英文)|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|A|A|B|B|無|無|D|D|


<br>
<br>

最後把aaa加到final

像是用aaa蓋上final重疊的部份
<br>
<br>
![Outerjoin](https://upload.cc/i1/2022/08/10/8DweRf.png "Outer join")




In [178]:
#用GRB替補MOST
final = final.replace({np.nan: None})

aaa = final[['GRB關鍵字(中文)', 'MOST關鍵字(中文)', 'GRB關鍵字(英文)','MOST關鍵字(英文)', 'GRB摘要(中文)', 'MOST摘要(中文)', 'GRB摘要(英文)', 'MOST摘要(英文)',]]


aaa = aaa.fillna(axis = 1 , method='ffill', limit=1)

aaa = aaa.loc[:,['MOST關鍵字(中文)','MOST關鍵字(英文)', 'MOST摘要(中文)', 'MOST摘要(英文)']]
aaa.columns =['關鍵字(中文)','關鍵字(英文)', '摘要(中文)', '摘要(英文)']



Final_final = pd.concat([final,aaa],axis =1 ,join = 'outer')

### 變項刪除及改名
<br>
保留需要的變項，同時改名


In [179]:
'''
cols = [
    '計畫主持人', '補', '助', '計', '畫', '編', '號', '機關名稱', '計畫名稱', '核定金額', '執行期限(起)',
    '執行期限(迄)', '學門', '職稱', 'Email', '學門代號', '人文及社會', '學門架構', '代碼加學門',
    '學門代碼\n(第3層)', '學門名稱(中)', '學門名稱(英)', 'GRB研究領域', 'ADD學校名稱', 'ADD系所名稱',
    'ADD機關地址', '原計畫編號', 'GRB系統辨識別', 'GRB系統編號', 'GRB原計畫編號', 'GRB調查計畫全稱(中)',
    'GRB調查計畫全稱(英)', 'GRB機關名稱', 'GRB計畫年份', 'GRB經費補助單位全稱(中)', 'GRB研究方式',
    'GRB研究性質', 'GRB研究領域(學門分類)', 'GRB計畫執行期間起', 'GRB計畫執行期間迄', 'GRB核定金額',
    'GRB計畫主持人', 'GRB共同主持人', 'GRB關鍵字(中文)', 'MOST關鍵字(中文)', 'GRB關鍵字(英文)',
    'MOST關鍵字(英文)', 'GRB摘要(中文)', 'MOST摘要(中文)', 'GRB摘要(英文)', 'MOST摘要(英文)',
    'GRB研究計畫詳目連結', 'MOST計畫主持人', 'MOST執行起訖', 'MOST機關名稱', 'MOST調查計畫全稱(中)',
    '計畫執行期間起', '計畫執行期間迄', '年期', '編號比對結果', '起始日期比對結果', '結束日期比對結果'
]
Final_final = Final_final[cols]  #--改名

'''



"\ncols = [\n    '計畫主持人', '補', '助', '計', '畫', '編', '號', '機關名稱', '計畫名稱', '核定金額', '執行期限(起)',\n    '執行期限(迄)', '學門', '職稱', 'Email', '學門代號', '人文及社會', '學門架構', '代碼加學門',\n    '學門代碼\n(第3層)', '學門名稱(中)', '學門名稱(英)', 'GRB研究領域', 'ADD學校名稱', 'ADD系所名稱',\n    'ADD機關地址', '原計畫編號', 'GRB系統辨識別', 'GRB系統編號', 'GRB原計畫編號', 'GRB調查計畫全稱(中)',\n    'GRB調查計畫全稱(英)', 'GRB機關名稱', 'GRB計畫年份', 'GRB經費補助單位全稱(中)', 'GRB研究方式',\n    'GRB研究性質', 'GRB研究領域(學門分類)', 'GRB計畫執行期間起', 'GRB計畫執行期間迄', 'GRB核定金額',\n    'GRB計畫主持人', 'GRB共同主持人', 'GRB關鍵字(中文)', 'MOST關鍵字(中文)', 'GRB關鍵字(英文)',\n    'MOST關鍵字(英文)', 'GRB摘要(中文)', 'MOST摘要(中文)', 'GRB摘要(英文)', 'MOST摘要(英文)',\n    'GRB研究計畫詳目連結', 'MOST計畫主持人', 'MOST執行起訖', 'MOST機關名稱', 'MOST調查計畫全稱(中)',\n    '計畫執行期間起', '計畫執行期間迄', '年期', '編號比對結果', '起始日期比對結果', '結束日期比對結果'\n]\nFinal_final = Final_final[cols]  #--改名\n\n"

In [180]:

'''
final['MOST關鍵字(中文)'] = final['GRB關鍵字(中文)'].fillna(method = 'backfill',limit=1, inplace = True)
final['GRB關鍵字(英文)'].fillna(method = 'backfill',limit=1, inplace = True)
final['GRB摘要(中文)'].fillna(method = 'backfill',limit=1, inplace = True)
final['GRB摘要(英文)'].fillna(method = 'backfill',limit=1, inplace = True)
'''
'''
final = final.drop(['機關名稱'], axis=1)
final = final.drop(['機關名稱_y'], axis=1)
final = final.drop(['補'], axis=1)
final = final.drop(['助'], axis=1)
final = final.drop(['計'], axis=1)
final = final.drop(['畫'], axis=1)
final = final.drop(['編'], axis=1)
final = final.drop(['號'], axis=1)
final = final.drop(['執行期限(起)'], axis=1)
final = final.drop(['執行期限(迄)'], axis=1)
final = final.drop(['代碼加學門'], axis=1)
final = final.drop(['學門代碼\n(第3層)'], axis=1)
final = final.drop(['系統辨識別'], axis=1)
final = final.drop(['系統編號'], axis=1)
final = final.drop(['核定金額_y'], axis=1)
final = final.drop(['計畫主持人_y'], axis=1)
final = final.drop(['機關名稱_x'], axis=1)
final = final.drop(['調查計畫全稱(中)_x'], axis=1)
final = final.drop(['關鍵字(中文)_y'], axis=1)
final = final.drop(['關鍵字(英文)_y'], axis=1)
final = final.drop(['摘要(中文)_y'], axis=1)
final = final.drop(['摘要(英文)_y'], axis=1)
final = final.drop(['執行起訖'], axis=1)
final = final.drop(['調查計畫全稱(中)_y'], axis=1)


#刪除計畫名稱一樣但年度不一樣的，保留第一筆(年度最接近者)
final = final.drop_duplicates(subset='計畫名稱',
                              keep='first',
                              inplace=False,
                              ignore_index=False)
'''
'''
final.columns = ['計畫主持人', '計畫名稱', '核定金額', '學門', '職稱', 'Email', '學門代號',
                 '人文及社會', '學門架構', '學門名稱(中)', 
                 

    
    
    '系統辨識別', '系統編號', '原計畫編號', '調查計畫全稱(中)', '調查計畫全稱(英)', '機關名稱', '計畫年份',
    '經費補助單位全稱(中)', '研究方式', '研究性質', '研究領域(學門分類)', '計畫執行期間起', '計畫執行期間迄',
    '核定金額(千元)', '計畫主持人', '共同主持人', '關鍵字(中文)', '關鍵字(英文)', '摘要(中文)', '摘要(英文)',
    '研究計畫詳目連結', '學校名稱', '系所名稱', '機關地址', '年期'
]
'''

print(Final_final)

    計畫主持人    補     助  計    畫    編    號                  機關名稱  \
0     施淑慎  108  2410  H    4  094  SSS          國立政治大學師資培育中心   
1     陳素燕  108  2410  H    7  045  SSS      國立清華大學學習科學與科技研究所   
2     辜玉旻  108  2410  H    8  039  SS2        國立中央大學學習與教學研究所   
3     林志哲  108  2410  H   27  008  SSS        國立臺北科技大學師資培育中心   
4      鄭雯  108  2410  H  110  021  SSS          國立中山大學師資培育中心   
..    ...  ...   ... ..  ...  ...  ...                   ...   
211   林妍如  108  2410  H  182  011  SS3          長庚大學人文及社會醫學科   
212   曾旭民  108  2410  H  182  007  SSS         長庚大學醫務管理學系（所）   
213   邱尚志  108  2410  H  227  004  SSS  國立臺北護理健康大學健康事業管理系（所）   
214   張振傑  108  2410  H  241  003  SSS      弘光科技大學健康事業管理系(所)   
215   洪春復  108  2410  H  507  001  SSS          國立金門大學長期照護學系   

                                         計畫名稱     核定金額  ...     比對 編號比對結果  \
0                        自我疼惜對大學生學習倦怠的緩解效果之探究   767000  ...  False  編號不一致   
1                      學習地圖、學習經驗與學習軌跡-高教新變革探究   881000  ...  False  編號不一致   


### 匯出檔案

In [181]:
Error = Final_final[Final_final['編號比對結果'].str.contains('不一致') == True]
Error = Final_final[Final_final['起始日期比對結果'].str.contains('不一致') == True]
Error = Final_final[Final_final['結束日期比對結果'].str.contains('不一致') == True]
Error = Final_final[Final_final['MOST缺失情況'].str.contains('程式在MOST查無資料，請手動查詢。') == True]
Error = Final_final[Final_final['GRB網站依據(缺失情況)'].str.contains('程式在GRB查無資料，請手動查詢。') == True]



Error = Error.loc[:, [
    '計畫主持人', '原計畫編號', 'GRB計畫編號', '計畫名稱', '執行期限(起)', '執行期限(迄)', 'MOST計畫執行期間起',
    'MOST計畫執行期間迄', '編號比對結果', '起始日期比對結果', '結束日期比對結果', 'MOST缺失情況','GRB網站依據(缺失情況)'
]]
error_Name = r'錯誤檔'  #儲存的檔名，可以不用更動

try:
    Error.to_excel(SaveFile_Path + '\\' + error_Name + "_" + "_" +
                   time.strftime("%m%d") + "_" + time.strftime("%H%M") +
                   ".xlsx",
                   encoding="utf_8_sig",
                   index=False)  #儲存結果
    print("錯誤檔匯出成功")  #最終檔名會加上 1.月份 2.日期 3.時間(時分)
except:
    print("錯誤檔匯出失敗")

Final_final = Final_final.drop(['補'], axis=1)
Final_final = Final_final.drop(['助'], axis=1)
Final_final = Final_final.drop(['計'], axis=1)
Final_final = Final_final.drop(['畫'], axis=1)
Final_final = Final_final.drop(['編'], axis=1)
Final_final = Final_final.drop(['號'], axis=1)
Final_final = Final_final.drop(['代碼加學門'], axis=1)
Final_final = Final_final.drop(['學門代碼\n(第3層)'], axis=1)
Final_final = Final_final.drop(['ADD系所名稱'], axis=1)
Final_final = Final_final.drop(['GRB全部計畫主持人'], axis=1)
Final_final = Final_final.drop(['GRB計畫編號'], axis=1)
Final_final = Final_final.drop(['GRB執行機構'], axis=1)
Final_final = Final_final.drop(['GRB研究領域_GRB'], axis=1)
Final_final = Final_final.drop(['GRB核定金額'], axis=1)
Final_final = Final_final.drop(['MOST缺失情況'], axis=1)
Final_final = Final_final.drop(['GRB研究期間'], axis=1)
Final_final = Final_final.drop(['GRB網站依據(缺失情況)'], axis=1)
Final_final = Final_final.drop(['GRB計畫主持人'], axis=1)
Final_final = Final_final.drop(['MOST計畫主持人'], axis=1)
Final_final = Final_final.drop(['MOST機關名稱'], axis=1)
Final_final = Final_final.drop(['MOST調查計畫全稱(中)'], axis=1)
Final_final = Final_final.drop(['MOST關鍵字(中文)'], axis=1)
Final_final = Final_final.drop(['MOST關鍵字(英文)'], axis=1)
Final_final = Final_final.drop(['MOST摘要(中文)'], axis=1)
Final_final = Final_final.drop(['MOST摘要(英文)'], axis=1)
Final_final = Final_final.drop(['GRB關鍵字(中文)'], axis=1)
Final_final = Final_final.drop(['GRB關鍵字(英文)'], axis=1)
Final_final = Final_final.drop(['GRB摘要(中文)'], axis=1)
Final_final = Final_final.drop(['GRB摘要(英文)'], axis=1)
Final_final = Final_final.drop(['比對'], axis=1)
Final_final = Final_final.drop(['編號比對結果'], axis=1)
Final_final = Final_final.drop(['起始日期比對'], axis=1)
Final_final = Final_final.drop(['起始日期比對結果'], axis=1)
Final_final = Final_final.drop(['結束日期比對'], axis=1)
Final_final = Final_final.drop(['結束日期比對結果'], axis=1)
Final_final = Final_final.drop(['MOST執行起訖'], axis=1)

#儲存結果
#★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
SaveFile_Path = r'C:\Users\user\Desktop\py\TEST'  #要儲存檔案的路徑
#★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

SaveFile_Name = r'合併'  #儲存的檔名，可以不用更動

try:
    Final_final.to_excel(SaveFile_Path + '\\' + SaveFile_Name + "_" + "_" +
                         time.strftime("%m%d") + "_" + time.strftime("%H%M") +
                         ".xlsx",
                         encoding="utf_8_sig",
                         index=False)  #儲存結果
    print("合併檔匯出成功")  #最終檔名會加上 1.月份 2.日期 3.時間(時分)
except:
    print("合併檔匯出失敗")

print("合併檔有" + str(len(Final_final)) + "筆資料")
print("錯誤檔有" + str(len(Error)) + "筆資料")

錯誤檔匯出成功
合併檔匯出成功
合併檔有216筆資料
錯誤檔有3筆資料
